In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
#Aqui primero lei varias reseñas para poder elegir las categorias

# Definir la función para categorizar reseñas
def categorizar_reseñas(review_text):
    # Convertir el texto a minúsculas para facilitar la comparación
    review_text = review_text.lower()

    # Categoría 1: Recomendación por buen servicio del conductor
    if any(word in review_text for word in ['friendly', 'prompt', 'arrived on time', 'professional', 'informed',
                                             'attentive', 'excellence', 'attention', 'arrived early', 'organized',
                                               'helpful', 'support', 'competent', 'serious', 'punctual',
                                                 'provide fast', 'arrive on time']):
        return "Recomendación por buen servicio del conductor"
    
    # Categoría 2: Recomendación por buen estado del vehículo
    elif any(word in review_text for word in ['clean', 'immaculate', 'maintained', 'impeccable', 'quiet', 
                                              'well maintained']):
        return "Recomendación por buen estado del vehículo"
    
    # Categoría 3: Queja por conductor
    elif any(word in review_text for word in ['late', 'had a bad attitude', 'thing were wrong', 'reckless',
                                               'Unprofessional', 'not punctual']):
        return "Queja por conductor"
    
    # Categoría 4: Queja por vehículo
    elif any(word in review_text for word in ['dirty', 'poor condition', 'noisy', 'bad condition', 'uncomfortable']):
        return "Queja por vehículo"
    
    # Si no se encuentra ninguna categoría específica, se devuelve "Otro"
    else:
        return "Otro"

# Aplicar la función a la columna 'review_english' y crear una nueva columna llamada 'category'
df_reseñas['category'] = df_reseñas['review_english'].apply(categorizar_reseñas)

# Verificar el DataFrame resultante
df_reseñas.head()



,Review Text,review_english,User Name,category
0,I cannot say enough good things about this car...,I cannot say enough good things about this car...,Julia S,Recomendación por buen servicio del conductor
1,Great service and easy to organize. Received ...,Great service and easy to organize. Received ...,Lilitreamont,Recomendación por buen servicio del conductor
2,We have used them 6 times this past year for m...,We have used them 6 times this past year for m...,BobnLaurie,Otro
3,"Great, Professional Drivers. They were early ...","Great, Professional Drivers. They were early ...",mld661,Recomendación por buen servicio del conductor
4,Great communication and service. Someone answe...,Great communication and service. Someone answe...,Audrey S,Recomendación por buen servicio del conductor


In [8]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
# Descargar recursos de NLTK
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Pruebo con arboles de decision 

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:

# Preprocesamiento de texto y vectorización
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df_reseñas['review_english'])
y = df_reseñas['category']

# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo de árbol de decisión
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(X_train, y_train)

# Predicciones sobre el conjunto de prueba
y_pred = tree_classifier.predict(X_test)

# Evaluación del modelo
print(classification_report(y_test, y_pred))


                                               precision    recall  f1-score   support

                                         Otro       0.77      0.79      0.78        29
                          Queja por conductor       1.00      0.67      0.80         3
   Recomendación por buen estado del vehículo       0.67      0.67      0.67         6
Recomendación por buen servicio del conductor       0.87      0.87      0.87        52

                                     accuracy                           0.82        90
                                    macro avg       0.82      0.75      0.78        90
                                 weighted avg       0.82      0.82      0.82        90



In [ ]:


# Uso del modelo para predecir nuevas reseñas
new_reviews = [
    "The taxi driver was very friendly and arrived on time. Excellent service!",
    "The vehicle was in poor condition and the driver was rude. Terrible experience.",
    "I highly recommend this service. The car was clean and the driver was professional.",
    "The driver was reckless and arrived late. I would not use this service again."
]

# Vectorización de las nuevas reseñas utilizando el vectorizador
X_new = vectorizer.transform(new_reviews)

# Predicciones utilizando el modelo de árbol de decisión
predictions = tree_classifier.predict(X_new)

# Imprimir las predicciones para las nuevas reseñas
print("Predicciones para nuevas reseñas:")
for review, prediction in zip(new_reviews, predictions):
    print(f"{review} -> {prediction}")


# Analisis de Sentimiento

In [14]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Descargo el léxico necesario para SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Creo una instancia de SentimentIntensityAnalyzer, que se utilizará para calcular la puntuación de polaridad de los textos.
sia = SentimentIntensityAnalyzer()

# Función para asignar etiquetas de sentimiento
def etiquetar_sentimiento(texto):
    if pd.notnull(texto):  # primero verifica si el valor no es nulo
        score = sia.polarity_scores(texto)['compound']
        #sia es la instancia creada al principio 
        #sia.polarity_scores(texto): Llama al método polarity_scores de SentimentIntensityAnalyzer y le pasa el texto como argumento. 
        # Este método devuelve un diccionario con varias puntuaciones, incluyendo la puntuación de compuesto.
        #['compound']: Accede al valor correspondiente a la clave 'compound' en el diccionario devuelto por polarity_scores. 
        # el valor obtenido es asignado a la variable score



        # Asigno etiquetas (0, 1, 2) a la puntuación de compuesto (score)
        if score >= 0.05:
            return 2  # Sentimiento positivo
        elif -0.05 < score < 0.05:
            return 1  # Sentimiento neutro
        else:
            return 0  # Sentimiento negativo
    else:
        return 1  # cuando es nulo retorna 1
    
# Aplico la función a la columna 'review_english' y creo con los resultados una nueva columna 'sentimiento'
df_reseñas['sentimiento'] = df_reseñas['review_english'].apply(etiquetar_sentimiento)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
#guardo el dataset con las columnas category y sentimiento

df_reseñas.to_csv('reviews.csv',index=False)